In [ ]:
import pandas as pd
from iminuit import Minuit 
from iminuit.cost import LeastSquares
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2


def retta (x,a,b):
    return (a*x + b)


def read():
    X = []
    S = []
    Z = pd.read_csv('F_Vb.CSV')
    for i in range(len(Z.columns.values)):
        if i != 0:
            z = Z.iloc[:, i]
            x = float(np.mean(z))
            if x >= 30:
                X.append(x)
                s = float(np.std(z))
                S.append(s)
    return (X, S)


def graph(X, S):
    Y = [52.5, 53, 53.5, 53.8, 54, 54.3, 54.5, 54.8, 55, 55.3, 55.5, 55.8, 56]
    Z = np.linspace(50, 56, 100000)
    fig, ax = plt.subplots(nrows = 1, ncols = 1)
    plt.plot(Z, retta(Z, 27.95, -1401))
    ax.scatter(Y,X)
    ax.errorbar(Y, X, S)
    plt.show()


def analisi(X, S):


    Y = [54, 54.3, 54.5, 54.8, 55, 55.3, 55.5, 55.8, 56]
    L_S = LeastSquares(Y, X, S, retta)
    my_minuit = Minuit (L_S, a = 0, b = 0)  
    my_minuit.migrad () 
    my_minuit.hesse ()   

#validità
    V = my_minuit.valid
    print('Validità: ', V)
    Q_squared = my_minuit.fval
    print ('Q-squared: ', Q_squared)
    N_dof = my_minuit.ndof
    print ('DOF: ', N_dof)
    p_v = 1 - chi2.cdf(Q_squared, N_dof)
    print('P value: ', p_v )

#valori dell'interpolazione
    a_f = my_minuit.values[0]
    b_f = my_minuit.values[1]

#visualizzazione 
    display(my_minuit)
    return (a_f, b_f)


S = read()
print(S)
'graph(S[0], S[1])'
Z = analisi(S[0], S[1])
print('a = ', Z[0])
print('b = ', Z[1])

In [ ]:
def intercetta():
    G = []
    X = np.linspace(40, 56, 100000000)
    for i in range(len(X)):
        y = float(retta(X[i], Z[0], Z[1]))
        if y <= 0.00001:
            if y >= -0.00001:
                G.append(X[i])
    print(np.max(G))
    print(np.mean(G))

intercetta()  


In [ ]:
def read1():
    X = []
    S = []
    Z = pd.read_csv('F_Vb.CSV')
    for i in range(len(Z.columns.values)):
        if i != 0:
            z = Z.iloc[:, i]
            x = float(np.mean(z))
            X.append(x)
            s = float(np.std(z))
            S.append(s)
    return (X, S)


def graph(X, S):
    Y = [52.5, 53, 53.5, 53.8, 54, 54.3, 54.5, 54.8, 55, 55.3, 55.5, 55.8, 56]
    Z = np.linspace(51.5, 56, 100000)
    fig, ax = plt.subplots(nrows = 1, ncols = 1)
    plt.plot(Z, retta(Z, 27.05, -1401))
    ax.scatter(Y, X, color = 'black')
    ax.errorbar(Y, X, S)
    plt.show()

In [ ]:
K = read1()
graph(K[0], K[1])
graph(np.sqrt(K[0]), K[1])